# Growth in GDP and EDU
This project contains data from the World Bank of growth in gross domestic product (GDP) and growth in expenditure on eduction (EDU) for a range of different countries. The objective of the project is to see whether there is a trend between these variables. 

To run the project you will need to import the packages below.

In [5]:
import pandas as pd                  # A data analysis tool
import numpy as np                   # Package for scientific computing with Python
import matplotlib.pyplot as plt      # A 2D plotting library to create beautiful figures
from matplotlib_venn import venn2    # An expansion in matplotlib to create venn diagrams 
import pandas_datareader             # Packages for reading data files
import datetime                      # Used to manipulate dates and times 
import seaborn as sns                # A library for making statistical graphics in Python
import statsmodels.formula.api as sm # A module needed to do OLS 
import os                            # To manipulate file paths
import ipywidgets as widgets         # To create interactive graphs 

To make the coding more simple and understandable we rename the file paths to GDP and EDU respectively.

In [6]:
GDP = 'GDPDATA.xlsx'
EDU = 'EducationDATA.xlsx'

##  Data cleaning on growth in expenditure on education (EDU)

The following part of the project will be focussing on cleaning the data file containing data on growth in expenditure on education (EDU). As you will see below there's a lot of unnecessary textcells and empty cells (NaN) which will need to be deleted in order to do an analysis on the data. To keep track of what we're doing with the data we will show the first five rows of the datafile. 

In [4]:
pd.read_excel(EDU).head(5)

,Data Source,World Development Indicators,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62
0,Last Updated Date,2019-03-21 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Country Name,Country Code,Indicator Name,Indicator Code,1960.0,1961.0,1962.0,1963.0,1964.0,1965.0,...,2009.00000,2010.00000,2011.00000,2012.00000,2013.00000,2014.00000,2015.00000,2016.00000,2017.00000,2018.0
3,Aruba,ABW,"Government expenditure on education, total (% ...",SE.XPD.TOTL.GD.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,5.92479,6.92927,6.11913,6.54906,6.80806,6.16293,6.47906,NaN,NaN,NaN
4,Afghanistan,AFG,"Government expenditure on education, total (% ...",SE.XPD.TOTL.GD.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,3.46196,3.43785,2.52441,3.43437,3.67390,3.24202,4.20525,3.92675,NaN


We start by deleting the first three rows of the dataset which mostly contains empty cells.

In [ ]:
edu = pd.read_excel(EDU, skiprows=3)
edu.head(5)

We delete the columns with data in the period of 1960-1998 since these years contains low amount of data. Furthermore, we remove redundant text cells. This leaves us with data from the years from 1999-2018 and the country code of each country.

In [ ]:
drop_these = ['Country Name', 'Indicator Name', 'Indicator Code', '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968', '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998']
edu.drop(drop_these, axis=1, inplace=True)
edu.head(5)


We also drop from the years in the period of 2016-2018 since not all countries have released data for these years yet.

In [ ]:
and_these = ['2018', '2017', '2016']
edu.drop(and_these, axis=1, inplace=True)
edu.head(5)

In [ ]:
edu.count() # To see how many columns there are 

We now drop all countries with more than two NaN's which leaves us with countries that contains a sufficient amount of data to analyse.

In [ ]:
edu = edu.dropna(thresh=18)
edu.head(5)

We then subset the data so the first column contains the contry code instead of numbers.

In [ ]:
edu = edu.set_index('Country Code')
edu.head(5)

We change the years from being variables with numbers to be variables with a letter (e for education). 

In [ ]:
myDict = {}
for i in range(1999, 2016): # our range is from 1999 to and including 2015 
    myDict[str(i)] = f'e{i}' 
myDict

And then we rename the years (so 1999 = e1999).

In [ ]:
edu.rename(columns = myDict, inplace=True)
edu.head(10)

We then switch the data from wide to long and rename the columns (e = edu) and (Country Code = Country_Code).

In [ ]:
edu = edu.reset_index()

In [ ]:
# Switching data from wide to long
edu = pd.wide_to_long(edu, stubnames='e', i='Country Code', j='Year')
edu.head(5)


In [ ]:
edu = edu.reset_index()
edu.rename(columns = {'e':'edu'}, inplace = True)
edu.rename(columns = {'Country Code':'Country_Code'}, inplace = True)
edu.head(5)

This leaves us with a datafile containing data for the years 1999-2015 and for countries with only a sufficient amount of observations.

## Plotting data for growth in expenditure on eduction (EDU)

In the following part of the project we will analyse the growth in EDU based on the cleaned datafile.

Right below is shown a scatter plot of the data.

In [ ]:
edu.plot.scatter(x = 'Year',y = 'edu')

We expected to see an increasing growth in EDU from year to year. Unfortunately, this doesn't seem to be the case as the growth in EDU seems to somewhat random. This might be due to the fact that there's so many different countries in the same scatterplot which makes it's confusing to see a clear relationship. 

To solve this we can create an interactive scatter plot of EDU for each seperate country which is shown below.

In [ ]:
def plot_edu(dataframe, Country_Code): 
    l = dataframe['Country_Code'] == Country_Code
    
    ax=dataframe.loc[l,:].plot(x='Year', y='edu', style='-o', legend='False')

In [ ]:
widgets.interact(plot_edu,
    dataframe = widgets.fixed(edu),
    Country_Code = widgets.Dropdown(description='Countries', options=edu.Country_Code.unique(), value='MEX')
); 


The interactive scatter plot shows a much more clear relationship for growth in EDU from year to year. As you scroll through the different countries, you'll realise that each country has developed in quite different ways. Take for example Finland (FIN) who went from a growth in EDU on 5.9% in 1999 to a growth on 7.1% in 2015 or Ireland (IRL) who went from a growth in EDU on 4.2% in 1999 to a growth on approxmately 3.8% in 2015. 

It's hard to explain the development each country has went through without further analysis. In the next part of the project we will try to explain this development in EDU by including data on growth in gross domestic product (GDP).

## Data cleaning on gross domestic product (GDP)

As we did with the data for growth in EDU, we will also clean the data for growth in GDP. The datasets are quite similiar so we won't show the first five rows after each operation. We start by delting the first three rows in the dataset.

In [ ]:
gdp = pd.read_excel(GDP, skiprows = 3)

As before, we remove the columns containing data in the period 1960-1999 and 2016-2018 as well as removing the columns containing indicator code, contry name and indicator name.

In [ ]:
Drop_these = ['Indicator Code','Country Name','Indicator Name','1960','1961','1962','1963','1964','1965','1966','1967','1968','1969','1970','1971','1972','1973','1974','1975','1976','1977','1978','1979','1980','1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998','2016','2017','2018']
gdp.drop(Drop_these, axis = 1, inplace = True) # Axis 1 = columns

We then remove all rows contaning empty values (NaN's).

In [ ]:
gdp = gdp.dropna()

And we index the data by country code.

In [ ]:
gdp = gdp.set_index('Country Code')

We rename the years (g1999 = 1999) to avoid any hiccups.

In [ ]:
MyDict1 = {}
for i in range(1999, 2016): 
    MyDict1[str(i)] = f'g{i}'
MyDict1

In [ ]:
gdp.rename(columns = MyDict1, inplace = True)

In [ ]:
gdp = gdp.reset_index()

And we switch the data from wide to long.

In [ ]:
gdp = pd.wide_to_long(gdp, stubnames='g', i='Country Code', j='Year')
gdp.head(5)

In [ ]:
gdp = gdp.reset_index()
gdp.rename(columns = {'g':'gdp'}, inplace = True)
gdp.rename(columns = {'Country Code':'Country_Code'}, inplace = True)
gdp.head(5)

This leaves us with a datafile for growth in gross domestic product containing the years in the period 1999-2015 for only countries with a sufficient amount of observations.

## Plotting data for growth in gross domestic product (GDP)

In the following part of the project we will analyse the growth in GDP based on the cleaned datafile.

Right below you'll see a scatter plot of the growth in GDP from year to year.

In [ ]:
gdp.plot.scatter(x = 'Year', y = 'gdp')

Just as with growth in EDU the scatter plot for all the countries containing data for growth in GDP is fairly confusing and it's hard to see a clear relationship. Therefore, we'll again create a scatter plot for each seperate country. 

In [ ]:
def plot_gdp(dataframe, Country_Code): 
    I = dataframe['Country_Code'] == Country_Code
    
    ax=dataframe.loc[I,:].plot(x='Year', y='gdp', style='-o', legend='False')

In [ ]:
widgets.interact(plot_gdp,
    dataframe = widgets.fixed(gdp),
    Country_Code = widgets.Dropdown(description='Countries', options=gdp.Country_Code.unique(), value='MEX')
); 


The scatter plot illustrating growth in GDP for each seperate country shows a much more clear relationship. If we again focus on Finland and Ireland, we see that Finland had a very varying growthin the period 1999-2015 with growth rates ranging from -8% to 6% and Ireland having growth rates varying from -5% to 25%. Generally Iran have had increasing growth rates in GDP during the period while Finland have had somewhat decreasing growth rates. This could indicate a negative relationship between growth in GDP and EDU. 

This might just be a coincidence though. We can analyse this relationship further by merging the data which we'll do in the following part of the project.

## Merging the data files

We'll start merging the data.

In [ ]:
merged_data = pd.merge(edu,gdp, on=['Country_Code','Year'], how='inner')
merged_data.head(5)

In [ ]:
merged_data.count()

The figure shown below illustrates how the data has been merged which is a called an inner merge. This type of merging makes it so that we only include data which is part of both datafiles. 

In [ ]:
plt.figure(figsize=(10,5))
v = venn2(subsets = (4, 4, 10), set_labels = ('GDP', 'EDU'))
v.get_label_by_id('100').set_text('Dropped'); v.get_patch_by_id('100').set_alpha(0.15)
v.get_label_by_id('010').set_text('Dropped'); v.get_patch_by_id('010').set_alpha(0.15)
v.get_label_by_id('110').set_text('Included')
plt.title('Inner join')
plt.show()

## Plotting the merged data

The data files have now been merged into a new combined data file which we can analyse. Firstly, we'll make a scatter plot showing the relationship between growth in EDU and growth in GDP.

In [ ]:
y = merged_data['gdp']
x = merged_data['edu']
plt.scatter(x, y, s=10)

z = np.polyfit(x, y, 1)
p = np.poly1d(z)
plt.plot(x,p(x),"r--")
plt.xlabel("edu")
plt.ylabel("gdp")

plt.show()

Earlier we took at look at Finland and Ireland where we with great uncertainty said that there might be a negative relationship between growth in EDU and GDP. The scatter plot of the merged data tells the same story. It's clear that there's a small but nonetheless negative relationship between growth in EDU and GDP. We can observe that countries with high growth in EDU generally have a lower growth in GDP compared to countries with low growth in EDU. 

We can analyse the data further by including a heatmap of the scatter plot above.

In [ ]:
sns.jointplot(x=merged_data.edu, y=merged_data.gdp, kind="hex").set_axis_labels('GDP', 'EDU')

From the heatmap we can observe that the majority of countries in the analysis have growth rates in GDP at about 5% and growth rates in EDU at about 1% (the darker the dots is, the more countries is centered around this point). 

We can extend the analysis of the merged data a bit further by creating scatter plots of the average one-year growth in the variables as shown below.

In [ ]:
ax = merged_data.groupby('Country_Code').agg(lambda x : x.pct_change().mean()).plot(kind='scatter',x='gdp',y='edu')
ax.set_xlabel('Avg. one-year growth in GDP');
ax.set_ylabel('Avg. one-year growth in EDU...');

Unfortunately, it's hard to get a clear view of what's happening in the scatter plot above. To solve this we can take the log average one-year growth of each variable which is shown below.

In [ ]:
ax = merged_data.groupby('Country_Code').agg(lambda x: (np.log(x)).diff().mean()).plot(kind='scatter',x='gdp',y='edu')
ax.set_xlabel('Avg. one-year log-difference in GDP');
ax.set_ylabel('Avg. one-year log-difference in CO2');

The scatter plot of the log differences in the variables shows a somewhat disorganised relationship. We can't conlude anything further from this.

As the last part of the analysis we have tried to create an OLS regression analysis which is shown below.

In [ ]:
logdiffs = merged_data.set_index('Country_Code').groupby(level=0).transform(lambda x : (np.log(x)).diff(1) ).reset_index()

In [ ]:
res0 = sm.ols(formula='edu~gdp',                    data=merged_data).fit()
res1 = sm.ols(formula='edu~gdp+C(Year)',            data=merged_data).fit()
res2 = sm.ols(formula='edu~gdp+C(Year)+C(Country_Code)', data=merged_data).fit()
res3 = sm.ols(formula='edu~gdp',                                    data=merged_data).fit() # in this dataframe, variables are already logged
res4 = sm.ols(formula='edu~gdp+C(Country_Code)',                         data=merged_data).fit() # in this dataframe, variables are already logged

print(f'Baseline:        {res0.params["gdp"] : 8.4f}')
print(f'Year FE:         {res1.params["gdp"] : 8.4f}')
print(f'Year+Country FE: {res2.params["gdp"] : 8.4f}')
print(f'FD:              {res3.params["gdp"]         : 8.4f}')
print(f'FD + country FE: {res4.params["gdp"]         : 8.4f}')

In [ ]:
res = pd.DataFrame(data = [
        ['Baseline',         res0.params["gdp"], res1.bse["gdp"]], 
        ['Year FE',        res1.params["gdp"], res1.bse["gdp"]], 
        ['Year+country FE', res2.params["gdp"], res2.bse["gdp"]], 
        ['FD',              res3.params["gdp"],         res3.bse["gdp"]],
        ['FD+country FE',   res4.params["gdp"],         res4.bse["gdp"]],
    ], columns=['Model','estimate','se'])

In [ ]:
# Regression analysis
fig, ax = plt.subplots()
x_pos = range(res.shape[0])
ax.bar(x_pos, res.estimate, yerr=res.se,  alpha=0.5, ecolor='black', capsize=5);
ax.set_xticks(x_pos);
ax.set_xticklabels(res.Model);
ax.set_ylabel('Estimate');
ax.set_xlabel('Model');

From the regression analysis we can you see how the different variables have contributed to the negative relationship between EDU and GDP. 

## Conclusion

The growth in expenditure on education (EDU) and the growth in gross domestic product (GDP) seems to vary a lot from country to country, where some countries have had decreasing growth in both variables and others have had increasing growth. From the merged data we can observe that there seems to be a negative relationship between the variables. This indicates that countries generally spend a bigger portion of GDP on education during rough economic periods. This might be due to the fact that countries generally spends the same amount of dollars on education each year - this notwithstanding, if GDP is growing or decreasing. Furthermore, domestical political decisions plays a huge role in how the relationship changes for each country.